Prepare features & labels


In [ ]:
# Handle NaN/Inf
all_data = all_data.replace([np.inf, -np.inf], np.nan)
all_data = all_data.fillna(all_data.mean())
all_data = all_data.fillna(0)

# Features
feature_cols = [c for c in all_data.columns if c not in ("Student_ID","PSY_label")]
X_df = all_data[feature_cols].astype(float).clip(-1e6, 1e6)

# Labels
if all_data['PSY_label'].notna().sum() >= 3:
    med = all_data['PSY_label'].median()
    y = (all_data['PSY_label'] > med).astype(int)
    print("Binarized PSY_label into High/Low by median:", med)
else:
    y = None
    print(" Not enough labels, supervised steps will be skipped.")


Binarized PSY_label into High/Low by median: 2.0884615384615386


Feature cleaning (variance + correlation)


In [ ]:
sel = VarianceThreshold(threshold=1e-5)
X_reduced = sel.fit_transform(X_df)
sel_features = [f for i,f in enumerate(X_df.columns) if sel.get_support()[i]]

X_reduced_df = pd.DataFrame(X_reduced, columns=sel_features)
corr = X_reduced_df.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [c for c in upper.columns if any(upper[c]>0.95)]
X_clean = X_reduced_df.drop(columns=to_drop)
pd.Series(X_clean.columns).to_csv("project/data/cleaned_feature_names.csv", index=False)

print("Final cleaned shape:", X_clean.shape)


Final cleaned shape: (38, 348)


Scale features

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)
joblib.dump(scaler, "project/models/scaler.pkl")
print("Scaled shape:", X_scaled.shape)

Scaled shape: (38, 348)
